In [146]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

# 2. Keypoints using MP Holistic

In [147]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

In [148]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [149]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS) # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [150]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [151]:
def extract_keypoints(results):
    pose = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose, lh, rh])

# 4. Setup Folders for Collection

In [152]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['hello','thanks','water','hungry','bathroom','iloveyou'])

# Thirty videos worth of data
no_sequences = 60

# Videos are going to be 30 frames in length
sequence_length = 30
# Folder start
start_folder = 60

In [153]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [154]:
label_map = {label:num for num, label in enumerate(actions)}

In [155]:
label_map

{'hello': 0,
 'thanks': 1,
 'water': 2,
 'hungry': 3,
 'yes': 4,
 'iloveyou': 5,
 'reapeat': 6,
 'no': 7,
 'forget': 8,
 'deaf': 9,
 'me': 10}

In [156]:
sequences, labels = [], []
for action in actions:
    for sequence in range(no_sequences):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [157]:
np.array(sequences).shape

(660, 30, 258)

In [158]:
np.array(labels).shape

(660,)

In [159]:
X = np.array(sequences)

In [160]:
X.shape

(660, 30, 258)

In [161]:
y = to_categorical(labels).astype(int)

In [162]:
y

array([[1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]])

In [163]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
#X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.9)
#X_valid, X_test, y_valid, y_test = train_test_split(X_rem,y_rem, test_size=0.05)

In [164]:
y_test.shape

(66, 11)

# 7. Build and Train LSTM Neural Network

In [177]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam

In [178]:
optim = Adam()

In [179]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)
#ES = EarlyStopping(monitor='loss',patience=50)

In [180]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,258)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))

model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [181]:
model.compile(optimizer= optim , loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [182]:
model.fit(X_train, y_train, epochs=1500, callbacks=[tb_callback ] )

Epoch 1/1500
19/19 [==============================] - 5s 67ms/step - loss: 2.3284 - categorical_accuracy: 0.0950
Epoch 2/1500
19/19 [==============================] - 1s 43ms/step - loss: 2.6556 - categorical_accuracy: 0.2670
Epoch 3/1500
19/19 [==============================] - 1s 41ms/step - loss: 1.9998 - categorical_accuracy: 0.2825
Epoch 4/1500
19/19 [==============================] - 1s 42ms/step - loss: 2.2777 - categorical_accuracy: 0.3147
Epoch 5/1500
19/19 [==============================] - 1s 42ms/step - loss: 1.6298 - categorical_accuracy: 0.3713
Epoch 6/1500
19/19 [==============================] - 1s 44ms/step - loss: 1.6165 - categorical_accuracy: 0.4595
Epoch 7/1500
19/19 [==============================] - 1s 40ms/step - loss: 1.7883 - categorical_accuracy: 0.3317
Epoch 8/1500
19/19 [==============================] - 1s 40ms/step - loss: 1.4162 - categorical_accuracy: 0.4611
Epoch 9/1500
19/19 [==============================] - 1s 41ms/step - loss: 1.0844 - categorical_

19/19 [==============================] - 1s 43ms/step - loss: 7.2137e-04 - categorical_accuracy: 1.0000
Epoch 144/1500
19/19 [==============================] - 1s 40ms/step - loss: 6.0830e-04 - categorical_accuracy: 1.0000
Epoch 145/1500
19/19 [==============================] - 1s 41ms/step - loss: 5.8560e-04 - categorical_accuracy: 1.0000
Epoch 146/1500
19/19 [==============================] - 1s 60ms/step - loss: 4.5907e-04 - categorical_accuracy: 1.0000
Epoch 147/1500
19/19 [==============================] - 1s 60ms/step - loss: 5.0018e-04 - categorical_accuracy: 1.0000
Epoch 148/1500
19/19 [==============================] - 1s 60ms/step - loss: 3.5497e-04 - categorical_accuracy: 1.0000
Epoch 149/1500
19/19 [==============================] - 1s 60ms/step - loss: 4.1951e-04 - categorical_accuracy: 1.0000
Epoch 150/1500
19/19 [==============================] - 1s 60ms/step - loss: 3.1540e-04 - categorical_accuracy: 1.0000
Epoch 151/1500
19/19 [==============================] - 1s 60ms

Epoch 212/1500
19/19 [==============================] - 1s 40ms/step - loss: 4.5863e-05 - categorical_accuracy: 1.0000
Epoch 213/1500
19/19 [==============================] - 1s 43ms/step - loss: 4.7649e-05 - categorical_accuracy: 1.0000
Epoch 214/1500
19/19 [==============================] - 1s 57ms/step - loss: 4.2103e-05 - categorical_accuracy: 1.0000
Epoch 215/1500
19/19 [==============================] - 1s 40ms/step - loss: 3.9414e-05 - categorical_accuracy: 1.0000
Epoch 216/1500
19/19 [==============================] - 1s 53ms/step - loss: 3.3299e-05 - categorical_accuracy: 1.0000
Epoch 217/1500
19/19 [==============================] - 1s 64ms/step - loss: 4.6292e-05 - categorical_accuracy: 1.0000
Epoch 218/1500
19/19 [==============================] - 1s 61ms/step - loss: 3.0477e-05 - categorical_accuracy: 1.0000
Epoch 219/1500
19/19 [==============================] - 1s 60ms/step - loss: 2.8183e-05 - categorical_accuracy: 1.0000
Epoch 220/1500
19/19 [==========================

19/19 [==============================] - 1s 60ms/step - loss: 4.4908e-06 - categorical_accuracy: 1.0000
Epoch 348/1500
19/19 [==============================] - 1s 59ms/step - loss: 3.8647e-06 - categorical_accuracy: 1.0000
Epoch 349/1500
19/19 [==============================] - 1s 39ms/step - loss: 3.5257e-06 - categorical_accuracy: 1.0000
Epoch 350/1500
19/19 [==============================] - 1s 40ms/step - loss: 2.9937e-06 - categorical_accuracy: 1.0000
Epoch 351/1500
19/19 [==============================] - 1s 40ms/step - loss: 4.6903e-06 - categorical_accuracy: 1.0000
Epoch 352/1500
19/19 [==============================] - 1s 40ms/step - loss: 3.9486e-06 - categorical_accuracy: 1.0000
Epoch 353/1500
19/19 [==============================] - 1s 40ms/step - loss: 3.2383e-06 - categorical_accuracy: 1.0000
Epoch 354/1500
19/19 [==============================] - 1s 40ms/step - loss: 4.7523e-06 - categorical_accuracy: 1.0000
Epoch 355/1500
19/19 [==============================] - 1s 40ms

Epoch 416/1500
19/19 [==============================] - 1s 79ms/step - loss: 1.3911e-06 - categorical_accuracy: 1.0000
Epoch 417/1500
19/19 [==============================] - 2s 106ms/step - loss: 1.8814e-06 - categorical_accuracy: 1.0000
Epoch 418/1500
19/19 [==============================] - 2s 121ms/step - loss: 2.0007e-06 - categorical_accuracy: 1.0000
Epoch 419/1500
19/19 [==============================] - 2s 97ms/step - loss: 1.3406e-06 - categorical_accuracy: 1.0000
Epoch 420/1500
19/19 [==============================] - 2s 79ms/step - loss: 1.4508e-06 - categorical_accuracy: 1.0000
Epoch 421/1500
19/19 [==============================] - 2s 81ms/step - loss: 1.7080e-06 - categorical_accuracy: 1.0000
Epoch 422/1500
19/19 [==============================] - 1s 68ms/step - loss: 1.2738e-06 - categorical_accuracy: 1.0000
Epoch 423/1500
19/19 [==============================] - 1s 64ms/step - loss: 1.2306e-06 - categorical_accuracy: 1.0000
Epoch 424/1500
19/19 [========================

...

Epoch 826/1500
19/19 [==============================] - 1s 43ms/step - loss: 1.1194e-08 - categorical_accuracy: 1.0000
Epoch 827/1500
19/19 [==============================] - 1s 54ms/step - loss: 1.3090e-08 - categorical_accuracy: 1.0000
Epoch 828/1500
19/19 [==============================] - 1s 63ms/step - loss: 1.2769e-08 - categorical_accuracy: 1.0000
Epoch 829/1500
19/19 [==============================] - 1s 45ms/step - loss: 1.3776e-08 - categorical_accuracy: 1.0000
Epoch 830/1500
19/19 [==============================] - 1s 45ms/step - loss: 1.1397e-08 - categorical_accuracy: 1.0000
Epoch 831/1500
19/19 [==============================] - 1s 58ms/step - loss: 1.4174e-08 - categorical_accuracy: 1.0000
Epoch 832/1500
19/19 [==============================] - 1s 61ms/step - loss: 1.4699e-08 - categorical_accuracy: 1.0000
Epoch 833/1500
19/19 [==============================] - 1s 58ms/step - loss: 1.3097e-08 - categorical_accuracy: 1.0000
Epoch 834/1500
19/19 [==========================

19/19 [==============================] - 2s 100ms/step - loss: 4.2512e-09 - categorical_accuracy: 1.0000
Epoch 963/1500
19/19 [==============================] - 2s 116ms/step - loss: 3.4514e-09 - categorical_accuracy: 1.0000
Epoch 964/1500
19/19 [==============================] - 2s 114ms/step - loss: 4.0269e-09 - categorical_accuracy: 1.00000s - loss: 3.4997e-09 - cate
Epoch 965/1500
19/19 [==============================] - 2s 112ms/step - loss: 4.5367e-09 - categorical_accuracy: 1.0000
Epoch 966/1500
19/19 [==============================] - 2s 94ms/step - loss: 3.4205e-09 - categorical_accuracy: 1.0000
Epoch 967/1500
19/19 [==============================] - 2s 87ms/step - loss: 5.0425e-09 - categorical_accuracy: 1.0000
Epoch 968/1500
19/19 [==============================] - 1s 61ms/step - loss: 5.5218e-09 - categorical_accuracy: 1.0000
Epoch 969/1500
19/19 [==============================] - 1s 65ms/step - loss: 4.8286e-09 - categorical_accuracy: 1.0000
Epoch 970/1500
19/19 [=========

19/19 [==============================] - 1s 51ms/step - loss: 0.1908 - categorical_accuracy: 0.9499
Epoch 1102/1500
19/19 [==============================] - 1s 47ms/step - loss: 0.2176 - categorical_accuracy: 0.9306
Epoch 1103/1500
19/19 [==============================] - 1s 44ms/step - loss: 0.1823 - categorical_accuracy: 0.9419
Epoch 1104/1500
19/19 [==============================] - 1s 39ms/step - loss: 0.2314 - categorical_accuracy: 0.9213
Epoch 1105/1500
19/19 [==============================] - 1s 38ms/step - loss: 0.2510 - categorical_accuracy: 0.9407
Epoch 1106/1500
19/19 [==============================] - 1s 38ms/step - loss: 0.2368 - categorical_accuracy: 0.9163
Epoch 1107/1500
19/19 [==============================] - 1s 52ms/step - loss: 0.2087 - categorical_accuracy: 0.9272
Epoch 1108/1500
19/19 [==============================] - 1s 49ms/step - loss: 0.2045 - categorical_accuracy: 0.9290
Epoch 1109/1500
19/19 [==============================] - 1s 40ms/step - loss: 0.1955 - c

19/19 [==============================] - 1s 75ms/step - loss: 0.7165 - categorical_accuracy: 0.6895
Epoch 1242/1500
19/19 [==============================] - 1s 69ms/step - loss: 0.5919 - categorical_accuracy: 0.7731
Epoch 1243/1500
19/19 [==============================] - 1s 66ms/step - loss: 0.6068 - categorical_accuracy: 0.7669
Epoch 1244/1500
19/19 [==============================] - 1s 66ms/step - loss: 0.5506 - categorical_accuracy: 0.7749
Epoch 1245/1500
19/19 [==============================] - 1s 68ms/step - loss: 1.0622 - categorical_accuracy: 0.6729
Epoch 1246/1500
19/19 [==============================] - 1s 73ms/step - loss: 0.8815 - categorical_accuracy: 0.5887
Epoch 1247/1500
19/19 [==============================] - 1s 67ms/step - loss: 0.6907 - categorical_accuracy: 0.7584
Epoch 1248/1500
19/19 [==============================] - 1s 65ms/step - loss: 0.6409 - categorical_accuracy: 0.7855
Epoch 1249/1500
19/19 [==============================] - 1s 62ms/step - loss: 0.6878 - c

19/19 [==============================] - 1s 41ms/step - loss: 0.0729 - categorical_accuracy: 0.9822
Epoch 1382/1500
19/19 [==============================] - 1s 41ms/step - loss: 0.0481 - categorical_accuracy: 0.9812
Epoch 1383/1500
19/19 [==============================] - 1s 50ms/step - loss: 0.0813 - categorical_accuracy: 0.9833
Epoch 1384/1500
19/19 [==============================] - 2s 86ms/step - loss: 0.0389 - categorical_accuracy: 0.9881
Epoch 1385/1500
19/19 [==============================] - 2s 85ms/step - loss: 0.0539 - categorical_accuracy: 0.9869
Epoch 1386/1500
19/19 [==============================] - 1s 74ms/step - loss: 0.0524 - categorical_accuracy: 0.9888
Epoch 1387/1500
19/19 [==============================] - 1s 73ms/step - loss: 0.0520 - categorical_accuracy: 0.9881
Epoch 1388/1500
19/19 [==============================] - 1s 74ms/step - loss: 0.1375 - categorical_accuracy: 0.9661
Epoch 1389/1500
19/19 [==============================] - 1s 75ms/step - loss: 0.2538 - c

In [183]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_42 (LSTM)               (None, 30, 64)            82688     
_________________________________________________________________
lstm_43 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_44 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_42 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_43 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_44 (Dense)             (None, 11)                363       
Total params: 237,515
Trainable params: 237,515
Non-trainable params: 0
_______________________________________________

# 8. Make Predictions

In [184]:
res = model.predict(X_test)

In [185]:
actions[np.argmax(res[1])]

'hello'

In [186]:
actions[np.argmax(res[1])]

'hello'

In [187]:
actions[np.argmax(y_test[1])]

'hello'

# 9. Save Weights

In [188]:
model.save('action.h5')

In [135]:
del model

In [140]:
model.load_weights('action.h5')

In [189]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [190]:
yhat = model.predict(X_test)

In [191]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [192]:
multilabel_confusion_matrix(ytrue, yhat)

array([[[61,  2],
        [ 0,  3]],

       [[60,  0],
        [ 0,  6]],

       [[62,  1],
        [ 0,  3]],

       [[61,  0],
        [ 1,  4]],

       [[56,  0],
        [ 1,  9]],

       [[58,  0],
        [ 2,  6]],

       [[60,  1],
        [ 0,  5]],

       [[56,  1],
        [ 0,  9]],

       [[61,  0],
        [ 0,  5]],

       [[60,  0],
        [ 1,  5]],

       [[60,  0],
        [ 0,  6]]], dtype=int64)

In [193]:
accuracy_score(ytrue, yhat)

0.9242424242424242